In [1]:
# Import libs

from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import csv
import boto3

In [17]:
# Time series CSV Urls ( Updated twice daily )
conf_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
recov_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'

# Building Dataframes
conf_d  =pd.read_csv(conf_url)
death_d  =pd.read_csv(death_url)
recov_d  =pd.read_csv(recov_url)



In [18]:
conf_d.head()
#death_d.head()
#recov_d.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20
0,Anhui,Mainland China,31.82571,117.2264,1,9,15,39,60,70,...,733,779,830,860,889,910,934,950,962,973
1,Beijing,Mainland China,40.18238,116.4142,14,22,36,41,68,80,...,315,326,337,342,352,366,372,375,380,381
2,Chongqing,Mainland China,30.05718,107.8740,6,9,27,57,75,110,...,428,468,486,505,518,529,537,544,551,553
3,Fujian,Mainland China,26.07783,117.9895,1,5,10,18,35,59,...,239,250,261,267,272,279,281,285,287,290
4,Gansu,Mainland China,36.06110,103.8343,0,2,2,4,7,14,...,79,83,83,86,87,90,90,90,90,91


In [19]:
import datetime
date_txt = list(conf_d.columns)[-2]


date_time = date_txt.split(' ')
date_2 = date_time[0].split('/')

if len(date_2[0])==1:
    month_str='0'+date_2[0]
else:
    month_str=date_2[0]
if len(date_2[1])==1:
    day_str='0'+date_2[1]
else:
    day_str=date_2[1]
if len(date_2[2])==2:
    year_str='20'+date_2[2]
else:
    year_str=date_2
    
date_f = '{}-{}-{}'.format(year_str,month_str,day_str)

date_f
# 2010-01-01

'2020-02-16'

# Beta
## Scope:

* Total Confirmed
* Confirmed by Country/Region
* Total Deaths
* Deaths by Country/Region
* Total Recovered
* Recovered by Country/Region


In [20]:
# Total Confirmed

conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
conf_d_NumOnly['Total_Conf']=conf_d_NumOnly.iloc[:,-1]
tot_conf = conf_d_NumOnly['Total_Conf'].sum()
#conf_d_NumOnly.head()

In [21]:
tot_conf

73260

In [22]:
# Confirmed by Country
conf_d_country = conf_d_NumOnly.groupby(['Country/Region']).sum()['Total_Conf'].sort_values(ascending=False)

conf_d_country.head()
#conf_d_country.loc['Hong Kong']

Country/Region
Mainland China    72364
Others              454
Singapore            77
Japan                66
Hong Kong            60
Name: Total_Conf, dtype: int64

## Functions
### For getting TOTAL and DF by Country

In [23]:
def get_total(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]
    tot_conf = df['Total_Conf'].sum()
    
    return tot_conf

In [24]:
def get_occr_country(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]    
    df = df.groupby(['Country/Region']).sum()['Total_Conf'].sort_values(ascending=False)
    
    return df #returns DF

In [25]:
tot_conf = get_total(conf_d)
tot_death = get_total(death_d)
tot_recov = get_total(recov_d)
print(tot_conf)
print(tot_death)
print(tot_recov)


73260
1868
12583


In [26]:
occr_conf = get_occr_country(conf_d)
occr_death = get_occr_country(death_d)
occr_recov = get_occr_country(recov_d)
#print(occr_conf)
#print(occr_death)
#print(occr_recov)


# Compiling code 

In [27]:
# Import Libs
import pandas as pd
import boto3
import datetime
import time

# Functions
def get_total(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]
    tot_conf = df['Total_Conf'].sum()
    
    return tot_conf

def get_occr_country(df):
    #conf_d_NumOnly = conf_d.drop(['Lat','Long'], axis =1 )
    df['Total_Conf']=df.iloc[:,-1]    
    df = df.groupby(['Country/Region']).sum()['Total_Conf'].sort_values(ascending=False)
    
    return df #returns DF


def get_date(df):
    # Standar Date Format: 2010-11-31
    if list(df.columns)[-1]!='Total_Conf':
        date_txt = list(conf_d.columns)[-1]
    else:
        date_txt = list(conf_d.columns)[-2]
    # print(date_txt)
    date_time = date_txt.split(' ')
    date_2 = date_time[0].split('/')

    if len(date_2[0])==1:
        month_str='0'+date_2[0]
    else:
        month_str=date_2[0]
    if len(date_2[1])==1:
        day_str='0'+date_2[1]
    else:
        day_str=date_2[1]
    if len(date_2[2])==2:
        year_str='20'+date_2[2]
    else:
        year_str=date_2
    
    date_f = '{}-{}-{}'.format(year_str,month_str,day_str)
    return date_f



# Time series CSV Urls ( Updated twice daily )
#conf_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Confirmed.csv'
#death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Deaths.csv'
#recov_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/time_series/time_series_2019-ncov-Recovered.csv'

# Building Dataframes
conf_d  = pd.read_csv(conf_url)
death_d  =pd.read_csv(death_url)
recov_d  =pd.read_csv(recov_url)

# Getting Basic Variables for Dictionary
tot_conf = int(get_total(conf_d))
tot_death = int(get_total(death_d))
tot_recov = int(get_total(recov_d))

occr_conf = get_occr_country(conf_d)
occr_death = get_occr_country(death_d)
occr_recov = get_occr_country(recov_d)

print('Total Confirmed Cases: {}'.format(tot_conf))
print('Total Deaths: {}'.format(tot_death))
print('Total Recovery Cases: {}'.format(tot_recov))
#occr_conf.head()
#occr_death.head()
#occr_recov.head()




Total Confirmed Cases: 73260
Total Deaths: 1868
Total Recovery Cases: 12583


In [28]:
# Connecting to DynamoDb

a_data = pd.read_csv('accessKeys.csv')
a_key = a_data['Access key ID'][0]
a_S_key = a_data['Secret access key'][0]
region='us-east-2'

dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=a_key,
    aws_secret_access_key=a_S_key,
    region_name=region
)
dynamoTable = dynamodb.Table('Corona_latest')

types = {'Confirmed':conf_d, 'Deaths':death_d, 'Recoverys':recov_d}

for key in types:
    dic = {}
    tot_c = int(get_total((types[key])))
    date = get_date(types[key])
    dic['Date']=date+' '+key
    dic['Type']=key
    dic['Total Cases']=tot_c
    dic['By Country']=[]
    
    occr_country = get_occr_country(types[key])
    for index, value in occr_country.items():
        dic2 = {'Country':index, 'Cases':value}
        dic['By Country'].append(dic2)

    #dynamoTable.put_item(Item=dic)
    print('Item for {} uploaded'.format(key))
    time.sleep(5)


Item for Confirmed uploaded
Item for Deaths uploaded
Item for Recoverys uploaded


In [29]:
# Configuring Access Keys

a_data = pd.read_csv('accessKeys.csv')
a_key = a_data['Access key ID'][0]
a_S_key = a_data['Secret access key'][0]
region='us-east-2'


In [30]:
# Connecting to DynamoDb

a_data = pd.read_csv('accessKeys.csv')
a_key = a_data['Access key ID'][0]
a_S_key = a_data['Secret access key'][0]
region='us-east-2'

dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=a_key,
    aws_secret_access_key=a_S_key,
    region_name=region
)
dynamoTable = dynamodb.Table('Corona_latest')

dic = {'region': 'Total', 'confirmed':int(tot_conf), 'death':int(tot_death), 'recovered':int(tot_recov)}
dynamoTable.put_item(Item=dic)
print('Done Total')

result = pd.concat([occr_conf.rename("Confirmed"), occr_death.rename("Death"), occr_recov.rename("Recovered")],
                   axis=1, sort=False)
for index, row in result.iterrows():
    dic = {'region':index,'confirmed':int(row['Confirmed']),'death': int(row['Death']),'recovered': int(row['Recovered'])}
    dynamoTable.put_item(Item=dic)
    print('Done {}'.format(index))
    



Done Total
Done Mainland China
Done Others
Done Singapore
Done Japan
Done Hong Kong
Done Thailand
Done South Korea
Done Malaysia
Done Taiwan
Done Germany
Done Vietnam
Done Australia
Done US
Done France
Done Macau
Done United Arab Emirates
Done UK
Done Canada
Done Philippines
Done India
Done Italy
Done Spain
Done Russia
Done Sweden
Done Sri Lanka
Done Nepal
Done Finland
Done Egypt
Done Cambodia
Done Belgium


# Scanning DinamoDB

In [32]:
def get_names():
    #names = ""
    dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=a_key,
    aws_secret_access_key=a_S_key,
    region_name=region
    )
    dynamoTable = dynamodb.Table('Corona_latest')
    
    response = table.scan()
    print (response)
    
    
    nameCount == len(response['Items'])
    for idx, item in enumerate(response['Items']):
        names += item['NameId']
        if idx == nameCount - 2:
            names += " and "
        elif idx != nameCount - 1:
            names += ", "
    return names

In [33]:
a_data = pd.read_csv('accessKeys.csv')
a_key = a_data['Access key ID'][0]
a_S_key = a_data['Secret access key'][0]
region='us-east-2'


dynamodb = boto3.resource(
'dynamodb',
aws_access_key_id=a_key,
aws_secret_access_key=a_S_key,
region_name=region
)
dynamoTable = dynamodb.Table('Corona_latest')

response = dynamoTable.scan()

df = pd.DataFrame(response['Items']).set_index('region')
t_conf = int(df.loc['Total', 'confirmed'])
t_death = int(df.loc['Total', 'death'])
t_recov = int(df.loc['Total', 'recovered'])

print('total confirmed: {}'.format(t_conf))
print('total death: {}'.format(t_death))
print('total death: {}'.format(t_recov))



total confirmed: 73260
total death: 1868
total death: 12583


In [39]:
from boto3.dynamodb.conditions import Key, Attr


a_data = pd.read_csv('accessKeys.csv')
a_key = a_data['Access key ID'][0]
a_S_key = a_data['Secret access key'][0]
region='us-east-2'


dynamodb = boto3.resource(
'dynamodb',
aws_access_key_id=a_key,
aws_secret_access_key=a_S_key,
region_name=region
)
dynamoTable = dynamodb.Table('Corona_latest')

#response = dynamoTable.scan()

response = dynamoTable.query(
    KeyConditionExpression=Key('region').eq('Total')
)

recovered = response['Items'][0]['recovered']
confirmed = response['Items'][0]['confirmed']
death = response['Items'][0]['confirmed']
